In [1]:
from rich import print
from pathlib import Path
import scanpy as sc
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
import re
import anndata as ad
import scipy as sp
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix
import os

/dsk2/user/lit/miniforge3/envs/Scanpy/lib/python3.10/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
print(os.getcwd())
os.chdir("/dsk2/user/lit/WorkSpace/sc_TE/irescue/sEOAD/outputs/")
print(os.getcwd())

/dsk2/user/lit/WorkSpace/sc_TE/sEOAD/code

/dsk2/user/lit/WorkSpace/sc_TE/irescue/sEOAD/outputs

In [3]:
meta1=pd.read_csv("/dsk2/user/lit/WorkSpace/sc_TE/sEOAD/sEOAD_meta.csv")

In [4]:
samplelist=meta1['sample_title'].tolist()

In [5]:
adatas=[]

In [6]:
for id in samplelist : 
    adata=sc.read_10x_mtx(f"/dsk2/user/lit/WorkSpace/sc_TE/irescue/sEOAD/outputs/{id}/counts/",)
    
    adata.obs['Barcode1']=adata.obs_names.astype(str)
    adata.obs['SampleID']=id
    adata.obs['Barcode2']=adata.obs['SampleID'].astype(str)+'_'+adata.obs['Barcode1'].astype(str)
    
    adata.obs_names=adata.obs['Barcode2']
    adata.obs.index.names=['bc']    
    adatas.append(adata)

In [7]:
adata=sc.concat(adatas=adatas,join='inner',merge='same',)

In [8]:
meta1 = meta1.set_index("sample_title")
key = "SampleID"
adata.obs = adata.obs.join(meta1, on=key, how="left", rsuffix="_meta")

In [9]:
adata_gene=sc.read_h5ad("/dsk2/user/lit/WorkSpace/sc_TE/sEOAD/sEOAD_popV_q.h5ad")

In [10]:
meta_1=adata_gene.obs[['bc','pct_counts_mt','SampleID','Diagnosis','popv_prediction','popv_prediction_score',]]

In [11]:
meta_1['bc_1']=meta_1['SampleID'].astype(str)+'_'+meta_1['bc'].astype(str)

/tmp/ipykernel_2680369/1802611632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_1['bc_1']=meta_1['SampleID'].astype(str)+'_'+meta_1['bc'].astype(str)


In [12]:
meta_1 = meta_1.set_index("bc_1")
key = "bc"
adata.obs = adata.obs.join(meta_1, on=key, how="left", rsuffix="_meta")

In [13]:
col = "popv_prediction"
s = adata.obs[col].astype("string")
t = s.str.strip()
bad = (
    t.isna()
    | (t == "")
    | t.str.lower().isin({"nan", "none", "null", "na", "n/a"})
)
adata = adata[~bad.to_numpy(), :].copy()
adata.obs[col] = t[~bad].astype("string").values

In [14]:
adata.obs['cell_type']=adata.obs["popv_prediction"].astype(str)

In [15]:
adata

AnnData object with n_obs × n_vars = 97903 × 1269391
    obs: 'Barcode1', 'SampleID', 'Barcode2', 'tissue', 'disease_state', 'Sex', 'age_on_death', 'num', 'bc', 'pct_counts_mt', 'SampleID_meta', 'Diagnosis', 'popv_prediction', 'popv_prediction_score', 'cell_type'
    var: 'gene_ids', 'feature_types'

In [16]:
adata.obs.sample(3)

,Barcode1,SampleID,Barcode2,tissue,disease_state,Sex,age_on_death,num,bc,pct_counts_mt,SampleID_meta,Diagnosis,popv_prediction,popv_prediction_score,cell_type
bc,,,,,,,,,,,,,,,
NIH03_GAAGGAACACTTACAG,GAAGGAACACTTACAG,NIH03,NIH03_GAAGGAACACTTACAG,Hippocampus,Control,male,61,3,NIH03_GAAGGAACACTTACAG,0.126957,NIH03,Control,Mic_Immune,7,Mic_Immune
NIH12_TTAAGGTGTGTGCACG,TTAAGGTGTGTGCACG,NIH12,NIH12_TTAAGGTGTGTGCACG,Hippocampus,sEOAD,female,61,9,NIH12_TTAAGGTGTGTGCACG,4.835767,NIH12,AD,Oli,7,Oli
NIH16_TGGACCGGTGTGCACG,TGGACCGGTGTGCACG,NIH16,NIH16_TGGACCGGTGTGCACG,Prefrontal cortex,sEOAD,male,62,13,NIH16_TGGACCGGTGTGCACG,0.188768,NIH16,AD,Oli,7,Oli


In [17]:
adata.var.sample(3)

,gene_ids,feature_types
AluJr_dup13245,AluJr_dup13245,Gene Expression
L1ME1_dup25736,L1ME1_dup25736,Gene Expression
AluY_dup50408,AluY_dup50408,Gene Expression


In [18]:
adata.obs.index.names=['SampleID_Barcode']
adata.obs['popv_prediction']=adata.obs['popv_prediction'].astype(str)

In [19]:
adata.write_h5ad("/dsk2/user/lit/WorkSpace/sc_TE/irescue/sEOAD/sEOAD_irescue.h5ad", compression='gzip')